In [198]:
import numpy as np
from copy import deepcopy
from dataclasses import dataclass, field

In [45]:
def parse_input(path: str) -> list[list[tuple[int, int]]]:
    with open(path, "r") as f:
        lines = f.readlines()
    paths = []
    for line in lines:
        line = line.strip()
        path = []
        if line != "":
            points = line.split(" -> ")
            for point in points:
                [x, y] = [int(coord) for coord in point.split(",")]
                path.append((x, y))
        paths.append(path)
    return paths

In [46]:
test_input = parse_input("inputs/d14_test")
test_input

[[(498, 4), (498, 6), (496, 6)], [(503, 4), (502, 4), (502, 9), (494, 9)]]

In [47]:
input = parse_input("inputs/d14")

In [48]:
def coords_between(c1: int, c2: int) -> list[int]:
    if c1 < c2:
        return list(range(c1, c2 + 1))
    elif c2 < c1: 
        return list(range(c1, c2 - 1, -1))
    else:
        return [c1]

In [158]:
@dataclass 
class Cave:
    paths: list[list[tuple[int, int]]] = field(default_factory=list)
    with_floor: bool = False

    def __post_init__(self):
        min_x, max_x, min_y, max_y = self._bounding_box()
        self.min_x = min_x
        self.max_x = max_x
        self.min_y = min_y
        self.max_y = max_y

        self.offset_x = self.min_x
        self.grid = np.zeros((self.max_y + 1, self.max_x - self.offset_x + 1), np.int32)
        self._fill_grid()

        self.sand_units = 0

    
    def _bounding_box(self):
        min_x, min_y = self.paths[0][0]
        max_x, max_y = min_x, min_y
        for path in self.paths:
            for (x, y) in path:
                if x > max_x:
                    max_x = x
                elif x < min_x:
                    min_x = x
                if y > max_y:
                    max_y = y
                elif y < min_y:
                    min_y = y
        return min_x, max_x, min_y, max_y

    
    def _fill_grid(self):
        for path in self.paths:
            for curr_idx in range(1, len(path)):
                (prev_x, prev_y), (curr_x, curr_y) = path[curr_idx - 1], path[curr_idx]
                for x in coords_between(prev_x, curr_x):
                    for y in coords_between(prev_y, curr_y):
                        self.grid[y, x - self.offset_x] = -1

    
    def drop_sand(self) -> tuple[tuple[int, int], bool, bool, bool]:
        self.sand_units += 1
        id = self.sand_units

        x, y = 500 - self.offset_x, 0
        came_to_rest, spilled, clogged = False, False, self.grid[y, x] != 0
        while not came_to_rest and not spilled and not clogged:
            # drop one step below
            if y + 1 <= self.max_y and self.grid[y + 1, x] == 0:
                y += 1
            # drop below and to the left
            elif y + 1 <= self.max_y and self.grid[y + 1, x - 1] == 0:
                y += 1
                x -= 1
            # drop below and to the right
            elif y + 1 <= self.max_y and self.grid[y + 1, x + 1] == 0:
                y += 1
                x += 1
            elif y >= self.max_y or x > self.max_x or x < (self.min_x - self.offset_x):
                self.sand_units -= 1
                spilled = True
            else:
                came_to_rest = True
                clogged = self.grid[0, 500 - self.offset_x] != 0
                
        if not spilled and not clogged:
            self.grid[y, x] = id

        return ((x, y), came_to_rest, spilled, clogged)


    def drop_many(self, units: int):
        """Returns True if all units came to rest, otherwise returns False."""
        for _ in range(0, units):
            (_x, _y), _came_to_rest, spilled, clogged = self.drop_sand()
            if spilled or clogged:
                return False
        return True


    def reset_sand(self):
        self.sand_units = 0
        self.grid[self.grid > 0] = 0


In [159]:
test_cave = Cave(test_input)
test_cave.grid

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0, -1, -1],
       [ 0,  0,  0,  0, -1,  0,  0,  0, -1,  0],
       [ 0,  0, -1, -1, -1,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, -1,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1,  0]], dtype=int32)

In [160]:
test_cave

Cave(paths=[[(498, 4), (498, 6), (496, 6)], [(503, 4), (502, 4), (502, 9), (494, 9)]], with_floor=False)

In [161]:
test_cave.drop_sand()

((6, 8), True, False, False)

In [162]:
test_cave.drop_many(22)
test_cave.grid

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 22,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 20, 19, 21,  0,  0],
       [ 0,  0,  0,  0, -1, 17, 16, 18, -1, -1],
       [ 0,  0,  0, 23, -1, 14, 13, 15, -1,  0],
       [ 0,  0, -1, -1, -1, 11,  8, 12, -1,  0],
       [ 0,  0,  0,  0, 10,  6,  4,  7, -1,  0],
       [ 0,  0,  0,  9,  5,  2,  1,  3, -1,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1,  0]], dtype=int32)

In [163]:
test_cave.drop_sand()
test_cave.grid

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 22,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 20, 19, 21,  0,  0],
       [ 0,  0,  0,  0, -1, 17, 16, 18, -1, -1],
       [ 0,  0,  0, 23, -1, 14, 13, 15, -1,  0],
       [ 0,  0, -1, -1, -1, 11,  8, 12, -1,  0],
       [ 0,  0,  0,  0, 10,  6,  4,  7, -1,  0],
       [ 0, 24,  0,  9,  5,  2,  1,  3, -1,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1,  0]], dtype=int32)

In [164]:
test_cave.drop_sand()
test_cave.grid

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 22,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 20, 19, 21,  0,  0],
       [ 0,  0,  0,  0, -1, 17, 16, 18, -1, -1],
       [ 0,  0,  0, 23, -1, 14, 13, 15, -1,  0],
       [ 0,  0, -1, -1, -1, 11,  8, 12, -1,  0],
       [ 0,  0,  0,  0, 10,  6,  4,  7, -1,  0],
       [ 0, 24,  0,  9,  5,  2,  1,  3, -1,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1,  0]], dtype=int32)

In [165]:
test_cave.drop_sand()

((-1, 9), False, True, False)

In [166]:
cave = Cave(input)
cave.grid.shape

(171, 70)

In [167]:
def p1(cave: Cave):
    spilled = False
    while not spilled:
        _coords, _came_to_rest, spilled, _clogged = cave.drop_sand()
    return cave.sand_units

In [168]:
test_cave.reset_sand()
test_cave.grid

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0, -1, -1],
       [ 0,  0,  0,  0, -1,  0,  0,  0, -1,  0],
       [ 0,  0, -1, -1, -1,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, -1,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1,  0]], dtype=int32)

In [169]:
assert p1(test_cave) == 24

In [170]:
assert p1(cave) == 832

In [191]:
def p2(cave: Cave):
    paths = deepcopy(cave.paths)
    paths.append([(cave.min_x - 1000, cave.max_y + 2), (cave.max_x + 1000, cave.max_y + 2)])
    
    cave = Cave(paths, with_floor=True)
    clogged = False
    while not clogged:
        _coords, _came_to_rest, _spilled, clogged = cave.drop_sand()
    return cave.grid.max()

In [193]:
assert p2(test_cave) == 93

In [194]:
assert p2(cave) == 27601

27601